In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
%load_ext autoreload
%autoreload 2

In [2]:
from src.models.modules import *
from src.models.loss import L1_clamp_loss
from dataclasses import dataclass
import torch

@dataclass
class SDFTransformerConfig:
    dim_context: int
    dim_input: int
    num_outputs: int
    dim_output: int
    delta: float = 0.1
    num_inds: int = 32
    dim_hidden: int = 128
    num_heads: int = 4
    ln: bool = False

class SDFTransformer(nn.Module):
    def __init__(self, config: SDFTransformerConfig):
        super(SDFTransformer, self).__init__()
        self.config = config
        self.input_proj = nn.Sequential(nn.Linear(config.dim_input, config.dim_hidden), nn.ReLU())
        self.context_proj = nn.Sequential(nn.Linear(config.dim_context, config.dim_hidden), nn.ReLU())
        self.cross = MAB(config.dim_hidden, config.dim_hidden, config.dim_hidden, config.num_heads, ln=config.ln)
        self.enc = nn.Sequential(
            ISAB(config.dim_hidden, config.dim_hidden, config.num_heads, config.num_inds, ln=config.ln),
            ISAB(config.dim_hidden, config.dim_hidden, config.num_heads, config.num_inds, ln=config.ln)
        )
        self.dec = nn.Sequential(
            PMA(config.dim_hidden, config.num_heads, config.num_outputs, ln=config.ln),
            SAB(config.dim_hidden, config.dim_hidden, config.num_heads, ln=config.ln),
            SAB(config.dim_hidden, config.dim_hidden, config.num_heads, ln=config.ln),
        )
        self.regr = nn.Sequential(
            nn.ReLU(),
            nn.Linear(config.dim_hidden, config.dim_output),
            nn.Tanh()
        )

    def forward(self, context: torch.Tensor, x: torch.Tensor, labels: torch.Tensor = None):
        x = self.input_proj(x)          # [batch_size, num_outputs, dim_hidden]
        y = self.context_proj(context)  # [batch_size, num_outputs, dim_hidden]
        x = x.repeat(1, y.shape[1], 1)  # [batch_size, context_size, dim_hidden]
        y = self.cross(x, y)           # [batch_size, context_size, dim_hidden]
        y = self.enc(y)                 # [batch_size, context_size, dim_hidden]
        y = self.dec(y)                 # [batch_size, num_outputs, dim_hidden]
        y = self.regr(y)                # [batch_size, num_outputs, dim_output]

        loss = None
        if labels is not None:
            loss = L1_clamp_loss(y, labels, self.config.delta)
        return {'loss': loss, 'logits': y}

torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

config = SDFTransformerConfig(dim_context=4, dim_input=3, num_outputs=1, dim_output=1)
model = SDFTransformer(config).to(device)
print(device)

cuda


In [3]:
from src.models.dataset import LazySampleDataset
from pathlib import Path

project_dir = Path(os.path.abspath('')).resolve().parent
procesed_dir = project_dir / 'data' / 'processed'

train_files = list(procesed_dir.rglob('*_train.hdf5'))
val_files = list(procesed_dir.rglob('*_val.hdf5'))

train_dataset = LazySampleDataset(train_files)
val_dataset = LazySampleDataset(val_files)

In [4]:
from src.data.load_data import get_results_dir
from datetime import datetime

notebook_name = '2024_11_19_cross_first'
current_date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
folder_name = f"{notebook_name}-{current_date}"
result_dir = get_results_dir() / folder_name
result_dir.mkdir(parents=True, exist_ok=True)
print(result_dir)

C:\_prog\vm_shared\attention-sdf\results\2024_11_19_cross_first-2024-11-19-23-00-44


In [5]:
from transformers import Trainer, TrainingArguments

batch_size = 40
training_args = TrainingArguments(
    output_dir=result_dir / "results",
    eval_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=8,
    logging_dir=result_dir / "logs",
    logging_steps=10,
    weight_decay=0.01,
    save_total_limit=3,
    seed=42
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [6]:
trainer.train()
train_dataset.close()
val_dataset.close()

  0%|          | 0/120000 [00:00<?, ?it/s]

{'loss': 0.0135, 'grad_norm': 0.8312228322029114, 'learning_rate': 4.999583333333333e-05, 'epoch': 0.0}
{'loss': 0.0081, 'grad_norm': 0.6627960801124573, 'learning_rate': 4.999166666666667e-05, 'epoch': 0.0}
{'loss': 0.0075, 'grad_norm': 1.6644930839538574, 'learning_rate': 4.99875e-05, 'epoch': 0.0}
{'loss': 0.0068, 'grad_norm': 1.6269149780273438, 'learning_rate': 4.998333333333334e-05, 'epoch': 0.0}
{'loss': 0.0066, 'grad_norm': 0.39425796270370483, 'learning_rate': 4.997916666666667e-05, 'epoch': 0.0}
{'loss': 0.0089, 'grad_norm': 2.0791068077087402, 'learning_rate': 4.9975e-05, 'epoch': 0.0}
{'loss': 0.008, 'grad_norm': 2.3981473445892334, 'learning_rate': 4.997083333333333e-05, 'epoch': 0.0}
{'loss': 0.0115, 'grad_norm': 2.9984445571899414, 'learning_rate': 4.996666666666667e-05, 'epoch': 0.01}
{'loss': 0.0098, 'grad_norm': 2.9658360481262207, 'learning_rate': 4.99625e-05, 'epoch': 0.01}
{'loss': 0.0086, 'grad_norm': 0.6417171359062195, 'learning_rate': 4.995833333333333e-05, 'ep

  0%|          | 0/5000 [00:00<?, ?it/s]

{'eval_loss': 0.003253897652029991, 'eval_runtime': 143.6173, 'eval_samples_per_second': 1392.59, 'eval_steps_per_second': 34.815, 'epoch': 1.0}
{'loss': 0.0034, 'grad_norm': 0.33164581656455994, 'learning_rate': 4.374583333333334e-05, 'epoch': 1.0}
{'loss': 0.0035, 'grad_norm': 0.16388849914073944, 'learning_rate': 4.374166666666667e-05, 'epoch': 1.0}
{'loss': 0.0031, 'grad_norm': 0.8295050263404846, 'learning_rate': 4.3737500000000006e-05, 'epoch': 1.0}
{'loss': 0.0028, 'grad_norm': 0.16871193051338196, 'learning_rate': 4.373333333333334e-05, 'epoch': 1.0}
{'loss': 0.0028, 'grad_norm': 0.37508660554885864, 'learning_rate': 4.372916666666667e-05, 'epoch': 1.0}
{'loss': 0.0033, 'grad_norm': 1.0452104806900024, 'learning_rate': 4.3725000000000006e-05, 'epoch': 1.0}
{'loss': 0.0037, 'grad_norm': 0.37238436937332153, 'learning_rate': 4.372083333333333e-05, 'epoch': 1.0}
{'loss': 0.0024, 'grad_norm': 0.6175239682197571, 'learning_rate': 4.371666666666667e-05, 'epoch': 1.01}
{'loss': 0.0037

  0%|          | 0/5000 [00:00<?, ?it/s]

...

{'eval_loss': 0.0019556188490241766, 'eval_runtime': 145.1893, 'eval_samples_per_second': 1377.512, 'eval_steps_per_second': 34.438, 'epoch': 6.0}
{'loss': 0.0016, 'grad_norm': 0.24451382458209991, 'learning_rate': 1.2495833333333335e-05, 'epoch': 6.0}
{'loss': 0.0031, 'grad_norm': 0.45310160517692566, 'learning_rate': 1.2491666666666668e-05, 'epoch': 6.0}
{'loss': 0.0019, 'grad_norm': 0.07480669766664505, 'learning_rate': 1.24875e-05, 'epoch': 6.0}
{'loss': 0.0025, 'grad_norm': 0.07068993896245956, 'learning_rate': 1.2483333333333335e-05, 'epoch': 6.0}
{'loss': 0.0023, 'grad_norm': 0.6955202221870422, 'learning_rate': 1.2479166666666667e-05, 'epoch': 6.0}
{'loss': 0.002, 'grad_norm': 0.28381818532943726, 'learning_rate': 1.2475e-05, 'epoch': 6.0}
{'loss': 0.0029, 'grad_norm': 0.7071631550788879, 'learning_rate': 1.2470833333333334e-05, 'epoch': 6.0}
{'loss': 0.0018, 'grad_norm': 0.2787240743637085, 'learning_rate': 1.2466666666666667e-05, 'epoch': 6.01}
{'loss': 0.0016, 'grad_norm': 0

  0%|          | 0/5000 [00:00<?, ?it/s]

{'eval_loss': 0.0019211848266422749, 'eval_runtime': 145.9236, 'eval_samples_per_second': 1370.58, 'eval_steps_per_second': 34.264, 'epoch': 7.0}
{'loss': 0.0024, 'grad_norm': 0.017364054918289185, 'learning_rate': 6.245833333333334e-06, 'epoch': 7.0}
{'loss': 0.0015, 'grad_norm': 0.10669074207544327, 'learning_rate': 6.241666666666667e-06, 'epoch': 7.0}
{'loss': 0.0018, 'grad_norm': 0.0250629261136055, 'learning_rate': 6.2375e-06, 'epoch': 7.0}
{'loss': 0.0019, 'grad_norm': 0.20828980207443237, 'learning_rate': 6.2333333333333335e-06, 'epoch': 7.0}
{'loss': 0.003, 'grad_norm': 0.34896692633628845, 'learning_rate': 6.229166666666667e-06, 'epoch': 7.0}
{'loss': 0.0023, 'grad_norm': 0.03756432980298996, 'learning_rate': 6.2250000000000005e-06, 'epoch': 7.0}
{'loss': 0.0024, 'grad_norm': 0.8509563207626343, 'learning_rate': 6.220833333333333e-06, 'epoch': 7.0}
{'loss': 0.0014, 'grad_norm': 0.17439007759094238, 'learning_rate': 6.2166666666666676e-06, 'epoch': 7.01}
{'loss': 0.0013, 'grad_

  0%|          | 0/5000 [00:00<?, ?it/s]

{'eval_loss': 0.0018178855534642935, 'eval_runtime': 144.3027, 'eval_samples_per_second': 1385.975, 'eval_steps_per_second': 34.649, 'epoch': 8.0}
{'train_runtime': 6939.4788, 'train_samples_per_second': 691.695, 'train_steps_per_second': 17.292, 'train_loss': 0.0025029551823157817, 'epoch': 8.0}


In [7]:
import json
from dataclasses import asdict

current_date = datetime.now().strftime("%Y-%m-%d")
model_name = f"{current_date}-model"
config_name = f"{current_date}-config.json"
trainer.save_model(result_dir / model_name)

with open(result_dir / config_name, 'w') as f:
    json.dump(asdict(config), f)

In [8]:
from src.visualization.generate_mesh import generate_mesh
from src.data.load_data import get_data_dir

obj_path = get_data_dir() / 'intermediate' / 'bunny' / 'stanford-bunny.obj'
infered_obj_path = result_dir / f"bunny-{current_date}-generalization.obj"
generate_mesh(model, obj_path, infered_obj_path, device, batch_size, resolution=100)

Processing batches:   0%|          | 0/25000 [00:00<?, ?it/s]

In [9]:
hdf5_path = get_data_dir() / 'processed' / 'bunny' / 'stanford-bunny_train.hdf5'
infered_obj_path = result_dir / f"{current_date}-train.obj"
generate_mesh(model, hdf5_path, infered_obj_path, device, batch_size, resolution=100)

Processing batches:   0%|          | 0/25000 [00:00<?, ?it/s]